In [1]:
import gc
import pandas as pd
import numpy as np
import pyarrow as pa
import tensorflow as tf
from pyarrow import parquet as pq
from collections import defaultdict

In [2]:
train = pd.read_csv('../Data/Train/sales_train_validation.csv')
train = train.drop(['item_id','dept_id','cat_id','store_id','state_id'],axis =1)
train.index = train.id
train = train.drop('id',axis = 1)

In [3]:
def _rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

def _get_time_tensor(arr,window_size):
    tmp = _rolling_window(arr,window_size+1)
    Xtensor = tmp[:,:-1]
    Ytensor = tmp[:,-1]
    return Xtensor,Ytensor

def np_to_time_tensor(arr,windowSize):
    if np.ndim(arr) > 1:
        for ix,v in enumerate(arr):
            yield _get_time_tensor(v,windowSize)
    else:
        yield _get_time_tensor(arr,windowSize) 

def _get_item_id(fullIndex):
    tmp = fullIndex.split('_')
    return '_'.join(tmp[:3])

def _get_store_id(fullIndex):
    tmp = fullIndex.split('_')
    return '_'.join(tmp[3:5])

def _get_cate_info(fullIndex,cateInfoDir):
    item_id = _get_item_id(fullIndex)
    store_id = _get_store_id(fullIndex)
    return pd.read_parquet(cateInfoDir,filters = [("item_id",'=',str(item_id)),("store_id",'=',str(store_id))])

def _get_events(calendar_dir,sdate,edate):
    df = pd.read_csv(calendar_dir)
    df['d_num'] = df.d.apply(lambda x: x.replace('d_','')).astype('int')
    return df[df.d_num.apply(lambda x: x <= edate and x >= sdate)]

def _get_future_events(calendar_dir, sdate,duration):
    edate = sdate+duration-1
    return _get_events(calendar_dir,sdate,edate)


In [107]:
x,y = np_to_time_tensor(train.head(1).values,100)

In [ ]:
x.shape

array([], dtype=float64)